# Calculate correlation of resting & reading

In [37]:
import pandas as pd
import numpy as np
from numpy import corrcoef
from scipy.signal import butter, lfilter, welch
import matplotlib.pyplot as plt

## Preparation

In [2]:
fs = 250

In [3]:
def high_pass(low, high, df, order=4):
    """
    Filter channel data with butterworth filter
    """
    nyq = 0.5 * fs
    b, a = butter(order, [low / nyq, high / nyq], btype='band')
    # filtered dataframe
    fil_df = pd.DataFrame(dtype="float64")
    for i, col in enumerate(df):
        fil_df[df.columns[i]] = lfilter(b, a, df[col])
    
    return fil_df

In [7]:
# try a file
rest_df = pd.read_csv('Data/Resting/R600_RestingState.csv')
read_df = pd.read_csv('Data/Reading/R600_ND.csv')

# filter
high = 40
low = 0.5
fil_rest = high_pass(low, high, rest_df)
fil_read = high_pass(low, high, read_df)

# take one channel
rest_cz = rest_df.Cz
read_cz = read_df.Cz
# clip, start at 1000th data point as there seems to be a noise
# make sure it contains integer number of epochs
start = 750
end = min(len(rest_cz), len(read_cz))
end = end - (end % fs)
read_cz = read_cz[start:end]
rest_cz = rest_cz[start:end]
assert read_cz.shape == rest_cz.shape

## Baseline

In [40]:
def baseline(resting_cz, fs):
    """
    Using resting data to calculate baseline. For now, using channle CZ
    """
    resting_cz = resting_cz.values.reshape((int(len(resting_cz) / (2 * fs)), 2 * fs))
    # power list to store the power spectrum for each epoch
    freq = []
    powers = []
    for epoch in resting_cz:
        f, power = welch(epoch, nperseg=fs, noverlap=None)
        freq = f
        powers.append(power)
    powers = np.asarray(powers)
    # take the mean of power
    mean_power = np.mean(powers, axis=0)
    assert len(mean_power) == powers.shape[1]
    # get correlations between each epoch power, and mean power
    corrs = []
    for power in powers:
        corrs.append(corrcoef(power, mean_power)[0][1])
    
    print(co)

SyntaxError: unexpected EOF while parsing (<ipython-input-40-4522004f0657>, line 19)

In [39]:
baseline(rest_cz, fs)

[[1.         0.93747131]
 [0.93747131 1.        ]]


## Baseline calculation

In [5]:
def correlation(rest, read):
    # calculate baseline
    rest = rest.values
    rest = rest.reshape((int(len(rest) / fs), fs))
    rest_epochs = []
    for e in rest:
        rest_epochs.append(np.mean(e))
    # normalize
#     rest_min = min(rest_epochs)
#     rest_max = max(rest_epochs)
#     for i in range(len(rest_epochs)):
#         rest_epochs[i] = (rest_epochs[i] - rest_min) / (rest_max - rest_min)
    
    print(np.var(rest_epochs))
    
    base_value = np.mean(np.asarray([np.mean(i) for i in rest]))
    print(base_value)
    
    
correlation(rest_cz, None)

32254097395.348415
-1675167.1705874586
